In [2]:
%load_ext autoreload
%autoreload 2

import argparse
import inspect
import os
import time

import matplotlib.pyplot as plt

from jax import jit, lax, random
from jax.example_libraries import stax
import jax.numpy as jnp
from jax.random import PRNGKey
import jax

import numpyro
from numpyro import optim
import numpyro.distributions as dist
from numpyro.examples.datasets import MNIST, load_dataset
from numpyro.infer import SVI, Trace_ELBO

from types import SimpleNamespace

from numpyro.optim import Adagrad

from numpyro.contrib.einstein import RBFKernel
from mixture_guide_impl_source import MixtureGuidePredictive
from stein_impl_source import SteinVI

import vae_example


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import jax.random as random
from numpyro import param
import numpyro.distributions as dist
from copy import deepcopy

# A simple guide that initializes a parameter randomly
def simple_guide():
    # If this is the first time running, 'x' is sampled from Normal(0, 1)
    x = param("x", lambda key: [[dist.Normal(0, 1).sample(key)], [dist.Normal(0, 1).sample(key)]])




In [5]:

def local_trace(key):
    guide = deepcopy(simple_guide)
    with numpyro.handlers.seed(rng_seed=key), numpyro.handlers.trace() as mixture_trace:
        guide()

    init_params = {
    name: site["value"]
    for name, site in mixture_trace.items()
    if site.get("type") == "param"
    }
    return init_params

my_key = PRNGKey(seed=5)

local_trace(my_key)

{'x': [Array(-1.37987, dtype=float32)]}

In [12]:
a = jax.vmap(local_trace)(random.split(my_key))
jax.tree.flatten(a)

([Array([-1.3889432 ,  0.09030154], dtype=float32),
  Array([-1.3889432 ,  0.09030154], dtype=float32)],
 PyTreeDef({'x': [[*], [*]]}))